In [2]:
!pip install --no-cache-dir unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 17.1 MB/s eta 0:00:00


# COLLECT_DATA

In [3]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import os
from unidecode import unidecode

# Duong dan
URL = 'https://vnexpress.net'

def get_links(url):
	links = []
	html = urlopen(url).read().decode('utf-8')
	soup = BeautifulSoup(html, 'html.parser')

	urls = set(re.findall(r'href=["\'](https?://[^"\']+\.html)["\']', str(soup)))

	for url in urls:
		if url != URL and url.startswith(URL) and not url.startswith(URL + '/tac-gia') and url.endswith('.html'):
			links.append(url)
			#print(url)

	return links

def get_link_by_catgories():
	# Danh sách các chuyên mục
	categories = ['thoi-su', 'kinh-doanh', 'cong-nghe', 'khoa-hoc', 'suc-khoe',
				  'the-thao', 'giai-tri', 'phap-luat', 'giao-duc', 'doi-song', 'du-lich', 'tam-su']

	#categories = ['thoi-su']

	# Danh sách các đường dẫn bài báo
	article_urls = {category: [] for category in categories}

	# Lấy link từ từng chuyên mục
	for category in categories:
		for i in range(1, 20):  # Lấy 10 trang trong mỗi chuyên mục
			url = f'{URL}/{category}-p{i}'
			article_urls[category] += get_links(url)

		# Loại bỏ đường dẫn trùng nhau
		article_urls[category] = list(set(article_urls[category]))

		# Hiển thị số lượng đường dẫn thu thập được
		print('Số lượng đường dẫn đã thu thập:')
		print(category, len(article_urls[category]))
	return article_urls

def get_article_content(url):
	try:
		content = ''
		html = urlopen(url).read()
		soup = BeautifulSoup(html, 'html.parser')

		# Lấy phần chứa nội dung bài báo
		div_content = soup.select('.page-detail .container')
		if not div_content:
			return "Không tìm thấy nội dung bài báo."

		div_content = div_content[0]

		# Lấy phần mô tả
		# description = div_content.find_all('p', {'class': 'description'})
		# if description:
		# 	description = description[-1]
		# 	text_description = description.get_text(strip=True)
		# 	location = description.find('span', {'class': 'location-stamp'})
		#
		# 	if location:
		# 		content = text_description[len(location.get_text()):]
		# 	else:
		# 		content = text_description

		# Lấy phần mô tả
		title = soup.title.get_text(strip=True)

		name = re.sub(r"[^a-z0-9]", "", unidecode(title).lower())[:30]

		# Lấy phần chi tiết bài báo
		detail = div_content.find('article', {'class': 'fck_detail'})
		if detail:
			p_normal = detail.find_all('p', {'class': 'Normal'})
			for p in p_normal:
				p_text = p.get_text(strip=True)
				if p_text and not p_text.startswith('>>') and p_text[-1] in ['.', '!', '?']:
					content += " " + p_text

		return {"content": content, "name": name}

	except Exception as e:
		print(f"Lỗi khi lấy nội dung từ {url}: {e}")
		return ''

def split_sentences(text):
    """
    Tách nội dung bài viết thành các câu dựa trên dấu câu (., ?, !).
    Xử lý trường hợp viết tắt để tránh cắt câu sai.
    """
    abbr_patterns = [
        (r'(?<!\w)(TP)\.(?=\w)', r'\1-'),  # TP.HCM -> TP-HCM
        (r'(?<!\w)(v)\.(v)\.(?=\W)', r'\1-\2'),  # v.v. -> v-v
    ]
    
    for pattern, repl in abbr_patterns:
        text = re.sub(pattern, repl, text)

    # Thêm dấu cách sau dấu câu nếu bị thiếu trước chữ viết hoa
    text = re.sub(r'([.!?])([A-ZÀ-Ỹ])', r'\1 \2', text)

    # Thêm dấu cách sau dấu phẩy nếu bị thiếu
    text = re.sub(r'([,])([^\s])', r'\1 \2', text)

    # Tách câu dựa trên dấu câu
    sentences = re.split(r'(?<=[.!?])\s+', text)

    # Khôi phục lại các viết tắt
    sentences = [s.replace('TP-', 'TP.').replace('v-v', 'v.v.') for s in sentences]

    # Loại bỏ khoảng trắng dư thừa và trả về danh sách câu
    return [s.strip() for s in sentences if s.strip()]

def save_cleaned_articles():
	article_urls = get_link_by_catgories()
	# # Chữ cái in hoa tiếng Việt
	# uppercase = "AÀÁẢÃẠĂẰẮẲẴẶÂẦẤẨẪẬBCCDĐEÈÉẺẼẸÊỀẾỂỄỆFGHIÌÍỈĨỊJKLMMNOÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢPQRSTUÙÚỦŨỤƯỪỨỬỮỰVWXYỲÝỶỸỴZ"
	#
	# # Danh sách tất cả các câu đã làm sạch của tất cả các bài viết
	sentences = []

	for category in article_urls.keys():
		# Tạo folder chuyên mục để chứa các file bài viết
		current_path = f'/kaggle/working/{category}/'
		os.makedirs(current_path, exist_ok=True)
		print('Đang lưu catgory', category)

		# Lưu các bài viết thành từng file
		for i in range(len(article_urls[category])):
			data = get_article_content(article_urls[category][i])
			if isinstance(data, dict):  # Kiểm tra nếu data là dict
				content = data.get("content", "")
				name = data.get("name", "")
				if content != '':
					file_path = f'{current_path}/{name}.txt'

					# Tách bài viết thành từng câu
					lines = split_sentences(content)

					# Ghi từng câu trong bài viết vào file và lưu vào danh sách câu
					with open(file_path, 'w', encoding='utf-8') as f:
						for line in lines:
							f.write(line + '\n')
							sentences.append(line)


	# Loại bỏ các câu trùng lặp
	sentences = list(set(sentences))
	print('Tổng số câu:', len(sentences))

	# File ngữ liệu
	file_corpus = open('/kaggle/working/corpus_2.txt', 'w', encoding='utf-8')

	count = 0  # Biến đếm số câu

	for sent in sentences:
		# Xóa ký tự thừa (ký tự zero-width)
		sent = re.sub(r'\s+', ' ', sent)

		# Ghi câu vào file (loại bỏ khoảng trắng thừa)
		file_corpus.write(sent.strip() + '\n')
		count += 1  # Tăng biến đếm

	# Đóng file sau khi hoàn tất
	file_corpus.close()

	# In số lượng câu trong file ngữ liệu
	print(f"Số lượng câu trong file dữ liêu: {count}")

if __name__ == '__main__':
	save_cleaned_articles()



Số lượng đường dẫn đã thu thập:
thoi-su 553
Số lượng đường dẫn đã thu thập:
kinh-doanh 590
Số lượng đường dẫn đã thu thập:
cong-nghe 583
Số lượng đường dẫn đã thu thập:
khoa-hoc 578
Số lượng đường dẫn đã thu thập:
suc-khoe 584
Số lượng đường dẫn đã thu thập:
the-thao 583
Số lượng đường dẫn đã thu thập:
giai-tri 570
Số lượng đường dẫn đã thu thập:
phap-luat 559
Số lượng đường dẫn đã thu thập:
giao-duc 592
Số lượng đường dẫn đã thu thập:
doi-song 583
Số lượng đường dẫn đã thu thập:
du-lich 601
Số lượng đường dẫn đã thu thập:
tam-su 115
Đang lưu catgory thoi-su
Đang lưu catgory kinh-doanh
Đang lưu catgory cong-nghe
Đang lưu catgory khoa-hoc
Đang lưu catgory suc-khoe
Lỗi khi lấy nội dung từ https://vnexpress.net/benh-nhan-thoat-noi-dau-viem-khop-nho-ky-thuat-noi-soi-co-tay-4855383.html: HTTP Error 504: Gateway Timeout
Lỗi khi lấy nội dung từ https://vnexpress.net/cach-bao-ve-phoi-cho-nguoi-benh-hen-suyen-4860424.html: HTTP Error 504: Gateway Timeout
Lỗi khi lấy nội dung từ https://vnexpres

# MAKE ADD_NOISE.PY

In [2]:
%%writefile add_noise.py

import re
import numpy as np
from unidecode import unidecode

letters = list(
    "abcdefghijklmnopqrstuvwxyzáàảãạâấầẩẫậăắằẳẵặóòỏõọôốồổỗộơớờởỡợéèẻẽẹêếềểễệúùủũụưứừửữựíìỉĩịýỳỷỹỵđĐABCDEFGHIJKLMNOPQRSTUVWXYZÁÀẢÃẠÂẤẦẨẪẬĂẮẰẲẴẶÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÉÈẺẼẸÊẾỀỂỄỆÚÙỦŨỤƯỨỪỬỮỰÍÌỈĨỊÝỲỶỸỴ")
letters2 = list("abcdefghijklmnopqrstuvwxyz")

typo = {"ă": "aw", "â": "aa", "á": "as", "à": "af", "ả": "ar", "ã": "ax", "ạ": "aj", "ắ": "aws", "ổ": "oor", "ỗ": "oox",
        "ộ": "ooj", "ơ": "ow",
        "ằ": "awf", "ẳ": "awr", "ẵ": "awx", "ặ": "awj", "ó": "os", "ò": "of", "ỏ": "or", "õ": "ox", "ọ": "oj",
        "ô": "oo", "ố": "oos", "ồ": "oof",
        "ớ": "ows", "ờ": "owf", "ở": "owr", "ỡ": "owx", "ợ": "owj", "é": "es", "è": "ef", "ẻ": "er", "ẽ": "ex",
        "ẹ": "ej", "ê": "ee", "ế": "ees", "ề": "eef",
        "ể": "eer", "ễ": "eex", "ệ": "eej", "ú": "us", "ù": "uf", "ủ": "ur", "ũ": "ux", "ụ": "uj", "ư": "uw",
        "ứ": "uws", "ừ": "uwf", "ử": "uwr", "ữ": "uwx",
        "ự": "uwj", "í": "is", "ì": "if", "ỉ": "ir", "ị": "ij", "ĩ": "ix", "ý": "ys", "ỳ": "yf", "ỷ": "yr", "ỵ": "yj",
        "đ": "dd",
        "Ă": "Aw", "Â": "Aa", "Á": "As", "À": "Af", "Ả": "Ar", "Ã": "Ax", "Ạ": "Aj", "Ắ": "Aws", "Ổ": "Oor", "Ỗ": "Oox",
        "Ộ": "Ooj", "Ơ": "Ow",
        "Ằ": "AWF", "Ẳ": "Awr", "Ẵ": "Awx", "Ặ": "Awj", "Ó": "Os", "Ò": "Of", "Ỏ": "Or", "Õ": "Ox", "Ọ": "Oj",
        "Ô": "Oo", "Ố": "Oos", "Ồ": "Oof",
        "Ớ": "Ows", "Ờ": "Owf", "Ở": "Owr", "Ỡ": "Owx", "Ợ": "Owj", "É": "Es", "È": "Ef", "Ẻ": "Er", "Ẽ": "Ex",
        "Ẹ": "Ej", "Ê": "Ee", "Ế": "Ees", "Ề": "Eef",
        "Ể": "Eer", "Ễ": "Eex", "Ệ": "Eej", "Ú": "Us", "Ù": "Uf", "Ủ": "Ur", "Ũ": "Ux", "Ụ": "Uj", "Ư": "Uw",
        "Ứ": "Uws", "Ừ": "Uwf", "Ử": "Uwr", "Ữ": "Uwx",
        "Ự": "Uwj", "Í": "Is", "Ì": "If", "Ỉ": "Ir", "Ị": "Ij", "Ĩ": "Ix", "Ý": "Ys", "Ỳ": "Yf", "Ỷ": "Yr", "Ỵ": "Yj",
        "Đ": "Dd"}

region = {"ẻ": "ẽ", "ẽ": "ẻ", "ũ": "ủ", "ủ": "ũ", "ã": "ả", "ả": "ã", "ỏ": "õ", "õ": "ỏ", "i": "j"}
region2 = {"s": "x", "l": "n", "n": "l", "x": "s", "d": "gi", "S": "X", "L": "N", "N": "L", "X": "S", "Gi": "D",
           "D": "Gi"}

vowel = list("aeiouyáàảãạâấầẩẫậăắằẳẵặóòỏõọôốồổỗộơớờởỡợéèẻẽẹêếềểễệúùủũụưứừửữựíìỉĩịýỳỷỹỵ")

acronym = {"không": "ko", " anh": " a", "em": "e", "biết": "bít", "giờ": "h", "gì": "j", "muốn": "mún", "học": "hok",
           "yêu": "iu",
           "chồng": "ck", "vợ": "vk", " ông": " ô", "được": "đc", "tôi": "t",
           "Không": "Ko", " Anh": " A", "Em": "E", "Biết": "Bít", "Giờ": "H", "Gì": "J", "Muốn": "Mún", "Học": "Hok",
           "Yêu": "Iu",
           "Chồng": "Ck", "Vợ": "Vk", " Ông": " Ô", "Được": "Đc", "Tôi": "T", }

teen = {"ch": "ck", "ph": "f", "th": "tk", "nh": "nk",
        "Ch": "Ck", "Ph": "F", "Th": "Tk", "Nh": "Nk"}


def teen_code(sentence, pivot):
    random = np.random.uniform(0, 1, 1)[0]
    new_sentence = str(sentence)
    if random > pivot:
        for word in acronym.keys():
            if re.search(word, new_sentence):
                random2 = np.random.uniform(0, 1, 1)[0]
                if random2 < 0.5:
                    new_sentence = new_sentence.replace(word, acronym[word])
        for word in teen.keys():
            if re.search(word, new_sentence):
                random3 = np.random.uniform(0, 1, 1)[0]
                if random3 < 0.05:
                    new_sentence = new_sentence.replace(word, teen[word])
        return new_sentence
    else:
        return sentence


def add_noise(sentence, pivot1=0.9, pivot2=0.985):
    sentence = teen_code(sentence, 0.5)
    noisy_sentence = ""
    i = 0
    while i < len(sentence):
        if sentence[i] not in letters:
            noisy_sentence += sentence[i]
        else:
            random = np.random.uniform(0, 1, 1)[0]
            if random < pivot1:
                noisy_sentence += (sentence[i])
            elif random < pivot2:
                if sentence[i] in typo.keys() and sentence[i] in region.keys():
                    random2 = np.random.uniform(0, 1, 1)[0]
                    if random2 <= 0.4:
                        noisy_sentence += typo[sentence[i]]
                    elif random2 < 0.8:
                        noisy_sentence += region[sentence[i]]
                    elif random2 < 0.95:
                        noisy_sentence += unidecode(sentence[i])
                    else:
                        noisy_sentence += sentence[i]
                elif sentence[i] in typo.keys():
                    random3 = np.random.uniform(0, 1, 1)[0]
                    if random3 <= 0.6:
                        noisy_sentence += typo[sentence[i]]
                    elif random3 < 0.9:
                        noisy_sentence += unidecode(sentence[i])
                    else:
                        noisy_sentence += sentence[i]
                elif sentence[i] in region.keys():
                    random4 = np.random.uniform(0, 1, 1)[0]
                    if random4 <= 0.6:
                        noisy_sentence += region[sentence[i]]
                    elif random4 < 0.85:
                        noisy_sentence += unidecode(sentence[i])
                    else:
                        noisy_sentence += sentence[i]
                elif i < len(sentence) - 1:
                    if sentence[i] in region2.keys() and (i == 0 or sentence[i - 1] not in letters) and sentence[
                        i + 1] in vowel:
                        random5 = np.random.uniform(0, 1, 1)[0]
                        if random5 <= 0.9:
                            noisy_sentence += region2[sentence[i]]
                        else:
                            noisy_sentence += sentence[i]
                    else:
                        noisy_sentence += sentence[i]

            else:
                new_random = np.random.uniform(0, 1, 1)[0]
                if new_random <= 0.33:
                    if i == (len(sentence) - 1):
                        continue
                    else:
                        noisy_sentence += (sentence[i + 1])
                        noisy_sentence += (sentence[i])
                        i += 1
                elif new_random <= 0.66:
                    random_letter = np.random.choice(letters2, 1)[0]
                    noisy_sentence += random_letter
                else:
                    pass

        i += 1
    return noisy_sentence



Overwriting add_noise.py


In [35]:
%%writefile add_noise_2.py

import re
import numpy as np
from unidecode import unidecode

letters=list("abcdefghijklmnopqrstuvwxyzáàảãạâấầẩẫậăắằẳẵặóòỏõọôốồổỗộơớờởỡợéèẻẽẹêếềểễệúùủũụưứừửữựíìỉĩịýỳỷỹỵđABCDEFGHIJKLMNOPQRSTUVWXYZÁÀẢÃẠÂẤẦẨẪẬĂẮẰẲẴẶÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÉÈẺẼẸÊẾỀỂỄỆÚÙỦŨỤƯỨỪỬỮỰÍÌỈĨỊÝỲỶỸỴĐ")
letters2=list("abcdefghijklmnopqrstuvwxyz")

typo={"ă":"aw","â":"aa","á":"as","à":"af","ả":"ar","ã":"ax","ạ":"aj","ắ":"aws","ổ":"oor","ỗ":"oox","ộ":"ooj","ơ":"ow",
"ằ":"awf","ẳ":"awr","ẵ":"awx","ặ":"awj","ó":"os","ò":"of","ỏ":"or","õ":"ox","ọ":"oj","ô":"oo","ố":"oos","ồ":"oof",
"ớ":"ows","ờ":"owf","ở":"owr","ỡ":"owx","ợ":"owj","é":"es","è":"ef","ẻ":"er","ẽ":"ex","ẹ":"ej","ê":"ee","ế":"ees","ề":"eef",
"ể":"eer","ễ":"eex","ệ":"eej","ú":"us","ù":"uf","ủ":"ur","ũ":"ux","ụ":"uj","ư":"uw","ứ":"uws","ừ":"uwf","ử":"uwr","ữ":"uwx",
"ự":"uwj","í":"is","ì":"if","ỉ":"ir","ị":"ij","ĩ":"ix","ý":"ys","ỳ":"yf","ỷ":"yr","ỵ":"yj","đ":"dd",
"Ă":"Aw","Â":"Aa","Á":"As","À":"Af","Ả":"Ar","Ã":"Ax","Ạ":"Aj","Ắ":"Aws","Ổ":"Oor","Ỗ":"Oox","Ộ":"Ooj","Ơ":"Ow",
"Ằ":"AWF","Ẳ":"Awr","Ẵ":"Awx","Ặ":"Awj","Ó":"Os","Ò":"Of","Ỏ":"Or","Õ":"Ox","Ọ":"Oj","Ô":"Oo","Ố":"Oos","Ồ":"Oof",
"Ớ":"Ows","Ờ":"Owf","Ở":"Owr","Ỡ":"Owx","Ợ":"Owj","É":"Es","È":"Ef","Ẻ":"Er","Ẽ":"Ex","Ẹ":"Ej","Ê":"Ee","Ế":"Ees","Ề":"Eef",
"Ể":"Eer","Ễ":"Eex","Ệ":"Eej","Ú":"Us","Ù":"Uf","Ủ":"Ur","Ũ":"Ux","Ụ":"Uj","Ư":"Uw","Ứ":"Uws","Ừ":"Uwf","Ử":"Uwr","Ữ":"Uwx",
"Ự":"Uwj","Í":"Is","Ì":"If","Ỉ":"Ir","Ị":"Ij","Ĩ":"Ix","Ý":"Ys","Ỳ":"Yf","Ỷ":"Yr","Ỵ":"Yj","Đ":"Dd"}

region={"ẻ":"ẽ","ẽ":"ẻ","ũ":"ủ","ủ":"ũ","ã":"ả","ả":"ã","ỏ":"õ","õ":"ỏ","i":"j"}
region2={"s":"x","l":"n","n":"l","x":"s","d":"gi","S":"X","L":"N","N":"L","X":"S","Gi":"D","D":"Gi"}

vowel=list("aeiouyáàảãạâấầẩẫậăắằẳẵặóòỏõọôốồổỗộơớờởỡợéèẻẽẹêếềểễệúùủũụưứừửữựíìỉĩịýỳỷỹỵ")

acronym={"không":"ko"," anh":" a","em":"e","biết":"bít","giờ":"h","gì":"j","muốn":"mún","học":"hok","yêu":"iu",
         "chồng":"ck","vợ":"vk"," ông":" ô","được":"đc","tôi":"t",
         "Không":"Ko"," Anh":" A","Em":"E","Biết":"Bít","Giờ":"H","Gì":"J","Muốn":"Mún","Học":"Hok","Yêu":"Iu",
         "Chồng":"Ck","Vợ":"Vk"," Ông":" Ô","Được":"Đc","Tôi":"T",}

teen={"ch":"ck","ph":"f","th":"tk","nh":"nk",
      "Ch":"Ck","Ph":"F","Th":"Tk","Nh":"Nk"}

def teen_code(sentence,pivot):
    random = np.random.uniform(0,1,1)[0]
    new_sentence=str(sentence)
    if random>pivot:
        for word in acronym.keys():
            if re.search(word, new_sentence):
                random2 = np.random.uniform(0,1,1)[0]
                if random2 <0.5:
                    new_sentence=new_sentence.replace(word,acronym[word])
        for word in teen.keys(): 
            if re.search(word, new_sentence):
                random3 = np.random.uniform(0,1,1)[0]
                if random3 <0.05:
                    new_sentence=new_sentence.replace(word,teen[word])        
        return new_sentence
    else:
        return sentence
    

def add_noise(sentence, pivot1=0.94,pivot2=0.985):
    sentence=teen_code(sentence,0.5)
    noisy_sentence = ""
    i = 0
    while i < len(sentence):
        if sentence[i] not in letters:
            noisy_sentence+=sentence[i]
        else: 
            random = np.random.uniform(0,1,1)[0]   
            if random < pivot1:
                noisy_sentence+=(sentence[i])
            elif random<pivot2:
                if sentence[i] in typo.keys() and sentence[i] in region.keys():
                    random2=np.random.uniform(0,1,1)[0]
                    if random2<=0.4:
                        noisy_sentence+=typo[sentence[i]]
                    elif random2<0.8:
                        noisy_sentence+=region[sentence[i]]
                    elif random2<0.95 :
                        noisy_sentence+=unidecode(sentence[i])
                    else:
                        noisy_sentence+=sentence[i]
                elif sentence[i] in typo.keys():
                    random3=np.random.uniform(0,1,1)[0]
                    if random3<=0.6:
                        noisy_sentence+=typo[sentence[i]]
                    elif random3<0.9 :
                        noisy_sentence+=unidecode(sentence[i])                        
                    else:
                        noisy_sentence+=sentence[i]
                elif sentence[i] in region.keys():
                    random4=np.random.uniform(0,1,1)[0]
                    if random4<=0.6:
                        noisy_sentence+=region[sentence[i]]
                    elif random4<0.85 :
                        noisy_sentence+=unidecode(sentence[i])                        
                    else:
                        noisy_sentence+=sentence[i]
                elif i<len(sentence)-1 :
                    if sentence[i] in region2.keys() and (i==0 or sentence[i-1] not in letters) and sentence[i+1] in vowel:
                        random5=np.random.uniform(0,1,1)[0]
                        if random5<=0.9:
                            noisy_sentence+=region2[sentence[i]]
                        else:
                            noisy_sentence+=sentence[i]
                    else:
                        noisy_sentence+=sentence[i]

            else:
                new_random = np.random.uniform(0,1,1)[0]
                if new_random <=0.33:
                    if i == (len(sentence) - 1):
                        continue
                    else:
                        noisy_sentence+=(sentence[i+1])
                        noisy_sentence+=(sentence[i])
                        i += 1
                elif new_random <= 0.66:
                    random_letter = np.random.choice(letters2, 1)[0]
                    noisy_sentence+=random_letter
                else:
                    pass
      
        i += 1
    return noisy_sentence

Writing add_noise_2.py


# TRAIN_DATA

In [26]:
import gdown
import os

#file_id = "1-0BsyW6zc15bGM0kWYdncz9fmEPBF4XR"  # Thay FILE_ID bằng ID thực của file corpus.txt
file_id = "1cMI3Tp1sb9lpkQSyduxdUmE3_rWN7K3u"
output = "model.keras"  # Đổi tên file lưu về máy nếu cần

gdown.download(f"https://drive.google.com/uc?id={file_id}", output, quiet=False)

print(os.listdir())  # Liệt kê file trong thư mục hiện tại

Downloading...
From: https://drive.google.com/uc?id=1cMI3Tp1sb9lpkQSyduxdUmE3_rWN7K3u
To: /kaggle/working/model.keras
100%|██████████| 20.4M/20.4M [00:00<00:00, 130MB/s]

['model.keras', 'add_noise.py', '.virtual_documents', 'model.h5', 'corpus.txt', '__pycache__', 'model_train.keras', 'state.db', 'model.png']


In [1]:
!pip install --no-cache-dir unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 15.6 MB/s eta 0:00:00


In [3]:
from add_noise_2 import *
from nltk import ngrams
import tensorflow as tf
# import tensorflow
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, TimeDistributed, Dense,LSTM, Bidirectional
from tensorflow.keras.callbacks import Callback, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.utils import plot_model

# Adam = tensorflow.keras.optimizers.Adam
# Callback = tensorflow.keras.models.Callback
# ModelCheckpoint = tensorflow.keras.models.ModelCheckpoint
# Sequential = tensorflow.keras.models.Sequential
# Activation = tensorflow.keras.layers.Activation
# TimeDistributed = tensorflow.keras.layers.TimeDistributed
# LSTM = tensorflow.keras.layers.LSTM
# Dense = tensorflow.keras.layers.Dense
# Bidirectional = tensorflow.keras.layers.Dense
# plot_model = tensorflow.keras.utils

NGRAM = 5
MAXLEN = 39
BATCH_SIZE = 512
EPOCHS = 25
alphabet = ['\x00', ' '] + list('0123456789') + letters

def read_file(name='corpus'):
    f = open(f'/kaggle/working/{name}.txt', 'r', encoding='utf-8')
    data = [line[:-1] for line in f.readlines()]
    f.close()
    #print('Số câu dữ liệu:', data[:10])
    return data

# 5-gram chứa tối đa 39 ký tự
# thêm phần đệm "\x00" vào cuối 5-gram để bằng với độ dài tối đa
def encoder_data(text, maxlen=MAXLEN):

    x = np.zeros((maxlen, len(alphabet)))
    for i, c in enumerate(text[:maxlen]):
        x[i, alphabet.index(c)] = 1
    if i < maxlen - 1:
        for j in range(i + 1, maxlen):
            x[j, 0] = 1
    return x

def decoder_data(x):
    x = x.argmax(axis=-1)
    return ''.join(alphabet[i] for i in x)

#  5-grams
# Từ dài nhất 7 ký tự ( nghiêng )
def gen_ngrams(words, n=5):
    tokens = words.split()
    if len(tokens) < n:  # Kiểm tra nếu số từ nhỏ hơn n
        return []  # Trả về danh sách rỗng thay vì raise lỗi
    return list(ngrams(tokens, n))

def extract_phrases(data):
    # Tách câu thành các cụm từ dựa vào các ký tự không phải số và chữ cái
    phrases = []

    for text in data:
        for c in set(text):
            if re.match(r'\w', c) and c not in alphabet:
                uc = unidecode(c)
                if re.match(r'\w', uc) and uc not in alphabet:
                    text = re.sub(c, "", text)  # Xóa ký tự nếu không thể chuyển đổi
                else:
                    text = re.sub(c, uc, text)  # Chuyển thành dạng không dấu

        # Tách các cụm từ dựa trên các ký tự không phải chữ cái và số
        phrases += re.findall(r'\w[\w\s]+', text)

    return [p.strip() for p in phrases if len(p.split()) > 1]

def train_ngrams(data):
    phrases = extract_phrases(data)
    list_ngrams = []
    for p in phrases:
        if any(c not in alphabet for c in p):
            continue
        for ngr in gen_ngrams(p, NGRAM):
            if len(" ".join(ngr)) < MAXLEN:
                list_ngrams.append(" ".join(ngr))

    return list_ngrams

def generate_data(data, batch_size):
    cur_index = 0
    while True:
        x, y = [], []
        for i in range(batch_size):
            y.append(encoder_data(data[cur_index]))
            x.append(encoder_data(add_noise(data[cur_index],0.94,0.985)))
            cur_index += 1
            if cur_index > len(data)-1:
                cur_index = 0
        yield np.array(x), np.array(y)

# Xây dựng mạng nơ-ron
# Được chuyển thể từ kiến trúc seq2seq, có thể được sử dụng cho Dịch máy, Tóm tắt văn bản, Ghi chú hình ảnh ...
def seq2seq():
    # data = ['Vụ cháy không gây thương vong, song thuê bao Vinaphone trên địa bàn Tiền Giang hầu như bị tê liệt.Do đó, theo kế hoạch của TP HCM,dự án đường nốiTrần Quốc Hoàn - Cộng Hoà, dài 4 km, dẫn vào ga T3 cũng sẽ khai thác đồng bộ khi nơi này hoạt động từ cuối tháng 4.']
    data = read_file('corpus_2')
    list_ngrams = train_ngrams(data)

    '''
    data = ['Miền Bắc từ đầu tuần trời nhiều mây, mưa nhỏ cùng độ ẩm cao khiến nền đất']
    print(list_ngrams)
    => ['Miền Bắc từ đầu tuần', 'Bắc từ đầu tuần trời', 'từ đầu tuần trời nhiều', 'đầu tuần trời nhiều mây', 'mưa nhỏ cùng độ ẩm', 'nhỏ cùng độ ẩm cao', 'cùng độ ẩm cao khiến', 'độ ẩm cao khiến nền', 'ẩm cao khiến nền đất']

    print(encoder_data('Trần Chiến Công').shape)
    print(decoder_data(encoder_data('Trần Chiến Công')))
    => (39, 198)
        Trần Chiến Công
    '''
    input_shape=(MAXLEN, len(alphabet))
    encoder = LSTM(256, input_shape=(MAXLEN, len(alphabet)), return_sequences=True)
    decoder = Bidirectional(LSTM(256, return_sequences=True, dropout=0.2))
    model = Sequential()
    model.add(encoder)
    model.add(decoder)
    model.add(TimeDistributed(Dense(256)))
    model.add(Activation('relu'))
    model.add(TimeDistributed(Dense(len(alphabet))))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(learning_rate=0.001),
                  metrics=['accuracy'])

    model.summary()
    plot_model(model, to_file='/kaggle/working/model.png')

    #80% train 20% valid
    train_data, valid_data = train_test_split(list_ngrams, test_size=0.2, random_state=42)
    train_generator = generate_data(train_data, batch_size=BATCH_SIZE)
    validation_generator = generate_data(valid_data, batch_size=BATCH_SIZE)
    sample_batch = next(iter(train_generator))  # Lấy một batch đầu tiên
    checkpointer = ModelCheckpoint(filepath=os.path.join('/kaggle/working/model_train_2.keras'), save_best_only=True, verbose=1)
    model.fit(
                    train_generator,
                    steps_per_epoch=max(1, len(train_data) // BATCH_SIZE),
                    epochs=EPOCHS,
                    validation_data=validation_generator,
                    validation_steps=max(1, len(valid_data) // BATCH_SIZE),
                    callbacks=[checkpointer])

def train_model():
    #seq2seq()
    strategy = tf.distribute.MirroredStrategy()  # Dùng 2 GPU song song
    print(f"Using {strategy.num_replicas_in_sync} GPUs")

    with strategy.scope():
        input_shape = (MAXLEN, len(alphabet))
        encoder = tf.keras.layers.LSTM(256, input_shape=input_shape, return_sequences=True)
        decoder = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True, dropout=0.2))
        model = tf.keras.Sequential([
            encoder,
            decoder,
            tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(256)),
            tf.keras.layers.Activation('relu'),
            tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(len(alphabet))),
            tf.keras.layers.Activation('softmax')
        ])

        model.compile(
            loss='categorical_crossentropy',
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            metrics=['accuracy']
        )

    model.summary()
    plot_model(model, to_file='/kaggle/working/model_2.png')

    train_data, valid_data = train_test_split(train_ngrams(read_file('corpus_2')), test_size=0.2, random_state=42)
    train_generator = generate_data(train_data, batch_size=BATCH_SIZE)
    validation_generator = generate_data(valid_data, batch_size=BATCH_SIZE)

    checkpointer = tf.keras.callbacks.ModelCheckpoint(
        filepath='/kaggle/working/model_train_4.keras', save_best_only=True, verbose=1
    )

    model.fit(
        train_generator,
        steps_per_epoch=max(1, len(train_data) // BATCH_SIZE),
        epochs=EPOCHS,
        validation_data=validation_generator,
        validation_steps=max(1, len(valid_data) // BATCH_SIZE),
        callbacks=[checkpointer]
    )

if __name__ == '__main__':
    train_model()


Using 2 GPUs


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 39, 256)             │         465,920 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 39, 512)             │       1,050,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed (TimeDistributed)   │ (None, 39, 256)             │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation (Activation)              │ (None, 39, 256)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_1 (TimeDistributed) │ (None, 39, 198)             │          50,886 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_1 (Activation)            │ (None, 39, 198)             │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,698,758 (6.48 MB)

 Trainable params: 1,698,758 (6.48 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/25
3297/3297 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - accuracy: 0.8456 - loss: 0.7143
Epoch 1: val_loss improved from inf to 0.10908, saving model to /kaggle/working/model_train_4.keras
3297/3297 ━━━━━━━━━━━━━━━━━━━━ 515s 153ms/step - accuracy: 0.8456 - loss: 0.7142 - val_accuracy: 0.9719 - val_loss: 0.1091
Epoch 2/25
3297/3297 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - accuracy: 0.9738 - loss: 0.1013
Epoch 2: val_loss improved from 0.10908 to 0.08269, saving model to /kaggle/working/model_train_4.keras
3297/3297 ━━━━━━━━━━━━━━━━━━━━ 505s 153ms/step - accuracy: 0.9738 - loss: 0.1013 - val_accuracy: 0.9784 - val_loss: 0.0827
Epoch 3/25
3297/3297 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - accuracy: 0.9795 - loss: 0.0788
Epoch 3: val_loss improved from 0.08269 to 0.07132, saving model to /kaggle/working/model_train_4.keras
3297/3297 ━━━━━━━━━━━━━━━━━━━━ 505s 153ms/step - accuracy: 0.9795 - loss: 0.0788 - val_accuracy: 0.9813 - val_loss: 0.0713
Epoch 4/25
3297/3297 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/st

In [22]:
from add_noise_2 import *
from collections import Counter
from tensorflow.keras.models import load_model
import nltk
import numpy as np
import re

model = load_model("model_train_4.keras")

NGRAM = 5
MAXLEN = 39
alphabet = ['\x00', ' '] + list('0123456789') + letters

# 5-gram chứa tối đa 39 ký tự
# thêm phần đệm "\x00" vào cuối 5-gram để bằng với độ dài tối đa
def encoder_data(text, maxlen=MAXLEN):
    x = np.zeros((maxlen, len(alphabet)))
    for i, c in enumerate(text[:maxlen]):
        x[i, alphabet.index(c)] = 1
    if i < maxlen - 1:
        for j in range(i + 1, maxlen):
            x[j, 0] = 1
    return x

def decoder_data(x):
    x = x.argmax(axis=-1)
    return ''.join(alphabet[i] for i in x)

def nltk_ngrams(sentence, n, maxlen):
    list_ngrams = []
    list_words = sentence.split()
    num_words = len(list_words)

    if (num_words >= n):
        for ngram in nltk.ngrams(list_words, n):
            if len(' '.join(ngram)) <= maxlen:
                list_ngrams.append(ngram)
    else:
        list_ngrams.append(tuple(list_words))

    return list_ngrams

def guess(ngram):
    text = ' '.join(ngram)
    preds = model.predict(np.array([encoder_data(text)]), verbose=0)

    return decoder_data(preds[0]).strip('\x00')
    
def add_punctuation(text, corrected_text):
    list_punctuation = {}

    for (i, word) in enumerate(text.split()):
        if word[0] not in alphabet or word[-1] not in alphabet:
            # Dấu ở đầu chữ như " và (
            start_punc = ''
            for c in word:
                if c in alphabet:
                    break
                start_punc += c

            # Dấu ở sau chữ như ?!.,
            end_punc = ''
            for c in word[::-1]:
                if c in alphabet:
                    break
                end_punc += c
            end_punc = end_punc[::-1]

            # Lưu vị trí từ và dấu câu trong từ đó
            list_punctuation[i] = [start_punc, end_punc]

    # Thêm dấu vào vị trí các từ đã đánh dấu
    result = ''
    for (i, word) in enumerate(corrected_text.split()):
        if i in list_punctuation:
            result += (list_punctuation[i][0] + word + list_punctuation[i][1]) + ' '
        else:
            result += word + ' '
    
    return result.strip()


def _correct(text):
    # Xóa các ký tự đặc biệt
    new_text = re.sub(r'[^' + ''.join(alphabet) + ']', ' ', text)

    ngrams = list(nltk_ngrams(new_text, NGRAM, MAXLEN))
    guessed_ngrams = list(guess(ngram) for ngram in ngrams)
    candidates = [Counter() for _ in range(len(guessed_ngrams) + NGRAM - 1)]

    for nid, ngram in (enumerate(guessed_ngrams)):
        for wid, word in (enumerate(re.split('\s', ngram))):
            candidates[nid + wid].update([word])

    corrected_text = ' '.join(c.most_common(1)[0][0] for c in candidates if c)
    return add_punctuation(text, corrected_text)

def correct(text):
    # Xóa các ký tự đặc biệt
    new_text = re.sub(r'[^' + ''.join(alphabet) + ']', '', text)

    ngrams = list(nltk_ngrams(new_text, NGRAM, MAXLEN))
    guessed_ngrams = list(guess(ngram) for ngram in ngrams)

    # Tính toán kích thước tối đa cần thiết cho candidates
    max_candidates = len(guessed_ngrams)
    candidates = [Counter() for _ in range(max_candidates + NGRAM - 1)]

    for nid, ngram in enumerate(guessed_ngrams):
        words = re.split(r'\s', ngram)
        for wid, word in enumerate(words[:NGRAM]):  # Giới hạn số từ được xử lý
            if nid + wid < len(candidates):
                candidates[nid + wid].update([word])


    # Kiểm tra và xử lý trường hợp không có 
    if not any(candidates):
        return text  # Trả về văn bản gốc nếu không có 

    corrected_words = []
    for c in candidates:
        if c:
            most_common = c.most_common(1)
            if most_common:
                corrected_words.append(most_common[0][0])

    corrected_text = ' '.join(corrected_words)

    return add_punctuation(text, corrected_text)

if __name__ == "__main__": 
    seq1 = 'Tháng ba ở Hà Nooij, thời tiết Hà Nội muwa phùn, nồm ẩm, nhưng riêng lúc chuyển mùa. Nhieeuf loài hoa như sưa, ban, mộc miên (hoa gạo), bưởi, xoan cùng bung lở đep theo cách. Lá cây lộc vừng cũng chuyển sắc đỏ rực, taoj nên khúc giao mùa xao xuyến với người dan và du khách.'
    seq2 = 'Tooi ddang hocj môn Xử lys ngoon nguwx tự nhiên. '
    print(_correct((seq1)))

Tháng ba ở Hà Nội, thời tiết Hà Nội mưa phùn, nồm ẩm, nhưng riêng lúc chuyển mùa. Nhiều loài hoa như xưa, ban, mộc miên (hoa gạo), bưởi, xoan cùng bung lở đẹp theo cách. Lá cây lộc vừng cũng chuyển sắc đỏ rực, tạo nên khúc giao mùa sao xuyến với người dân và du khách.


In [15]:
import os

file_path = "/kaggle/working/model.keras"  # Đường dẫn đến file cần xóa

if os.path.exists(file_path):
    os.remove(file_path)
    print("Đã xóa file thành công!")
else:
print("File không tồn tại!")

IndentationError: expected an indented block after 'else' statement on line 8 (2740614773.py, line 9)

In [4]:
!pip list > library_versions.txt
!python --version >> library_versions.txt


In [6]:
!zip -r /kaggle/working/working_folder.zip /kaggle/working

  adding: kaggle/working/ (stored 0%)
  adding: kaggle/working/kinh-doanh/ (stored 0%)
  adding: kaggle/working/kinh-doanh/benxemientaydatmuctieulaikyluc.txt (deflated 54%)
  adding: kaggle/working/kinh-doanh/dexuattang38250tydongvonchoton.txt (deflated 60%)
  adding: kaggle/working/kinh-doanh/thutuongdenghispxemxetnanghang.txt (deflated 62%)
  adding: kaggle/working/kinh-doanh/botaichinhphaitrinhnghiquyetve.txt (deflated 58%)
  adding: kaggle/working/kinh-doanh/chungkhoanhomnay192loatcophieu.txt (deflated 49%)
  adding: kaggle/working/kinh-doanh/haidoanhnghiepviphamkhibancheo.txt (deflated 63%)
  adding: kaggle/working/kinh-doanh/nhatbanchotdientaitaovahatnhan.txt (deflated 50%)
  adding: kaggle/working/kinh-doanh/giatomhumchamdaynguoinuoilonan.txt (deflated 56%)
  adding: kaggle/working/kinh-doanh/botaichinhkhongdongydanhthuela.txt (deflated 58%)
  adding: kaggle/working/kinh-doanh/chungkhoanmygiammanhnhat18than.txt (deflated 52%)
  adding: kaggle/working/kinh-doanh/chutichsunshineho

In [8]:
import importlib.metadata

# Danh sách các thư viện cần lấy phiên bản
used_packages = [
    "collections", "tensorflow", "nltk", "numpy", "os", "sklearn",
    "keras", "unidecode", "bs4"
]

# Ghi phiên bản vào file requirements.txt
with open("/kaggle/working/requirements.txt", "w") as f:
    for pkg in used_packages:
        try:
            version = importlib.metadata.version(pkg)
            f.write(f"{pkg}=={version}\n")
        except importlib.metadata.PackageNotFoundError:
            print(f"⚠️ Không tìm thấy thư viện: {pkg}")

print("✅ Đã tạo requirements.txt chỉ chứa các thư viện đã sử dụng.")

/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:31: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


AttributeError: module 'pip' has no attribute '_internal'